# Create photoreduction forcing files

In [16]:
import numpy as np
import netCDF4 as nc
import xarray as xr

%matplotlib inline

In [17]:
imin, imax = 1099, 2179
jmin, jmax = 159, 799

In [18]:
mask  = nc.Dataset('/ocean/brogalla/GEOTRACES/ariane_runs/ANHA12_Ariane_mesh.nc')
tmask = np.array(mask.variables['tmask'])
cond = (tmask[0,:,:,:] > 0.1) 
land_mask = np.ma.masked_where(cond, tmask[0,:,:,:])

mesh  = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12-EXH006_5d_gridT_y2002m01d05.nc')
lons  = np.array(mesh.variables['nav_lon'])
lats  = np.array(mesh.variables['nav_lat'])
tmask = tmask[0,:,imin:imax,jmin:jmax]

Parameterization for photo reduction:
- Higher sol_flux means stronger photo reduction.
- Background reduction value: 1
- Photo-enhanced reduction value maximum: 58

# Create forcing files

In [19]:
def load_variables(filename):
    a = nc.Dataset(filename)
    
    sol_flux = np.array(a.variables['iocesflx'][:,0,:,:])     
    
    return sol_flux

In [20]:
def red_rate_calc(year):
    sol_flux = load_variables(f'/data/brogalla/ANHA12/icemod_ave/ANHA12_EXH006_{year}_monthly.nc')
    max_sol_flux  = sol_flux[:,imin:imax,jmin:jmax].max()
    mean_sol_flux = sol_flux[:,imin:imax,jmin:jmax].mean()
    
    normalized_solar_flux = np.abs(sol_flux - mean_sol_flux)/max_sol_flux
    red_rate = normalized_solar_flux * 58
    
    red_rate[red_rate < 1] = 1
    red_rate[red_rate > 58] = 58
    
    return red_rate

In [21]:
def save_file(year, red_multiplier):
    
    location='/data/brogalla/reduction_forcing/extended/'
    
    for month in range(1,13):
    
        file_write = xr.Dataset(
            {'kred': (("y","x"), red_multiplier[month-1,:,:])}, 
            coords = {
                "y": np.zeros(2400),
                "x": np.zeros(1632),
            })

        file_write.to_netcdf(f'{location}red_y{year}m{month:02}.nc')
    
    return

In [22]:
for year in range(2002,2020):
    print(f'Calculation reduction rate multipliers for {year}')
    red_multiplier_year = red_rate_calc(year)
    save_file(year, red_multiplier_year)

Calculation reduction rate multipliers for 2002
Calculation reduction rate multipliers for 2003
Calculation reduction rate multipliers for 2004
Calculation reduction rate multipliers for 2005
Calculation reduction rate multipliers for 2006
Calculation reduction rate multipliers for 2007
Calculation reduction rate multipliers for 2008
Calculation reduction rate multipliers for 2009
Calculation reduction rate multipliers for 2010
Calculation reduction rate multipliers for 2011
Calculation reduction rate multipliers for 2012
Calculation reduction rate multipliers for 2013
Calculation reduction rate multipliers for 2014
Calculation reduction rate multipliers for 2015
Calculation reduction rate multipliers for 2016
Calculation reduction rate multipliers for 2017
Calculation reduction rate multipliers for 2018
Calculation reduction rate multipliers for 2019


Check a file:

In [23]:
location='/data/brogalla/reduction_forcing/extended/'
file='red_y2002m09.nc'
ds = nc.Dataset(f'{location}{file}')
reduction=np.array(ds.variables['kred'])

In [24]:
print(np.amax(reduction), np.amin(reduction), np.average(reduction))

58.0 1.0 13.190254
